In [16]:
# Adapted from https://github.com/GoogleCloudPlatform/cloudml-samples/blob/master/notebooks/tensorflow/getting-started-keras.ipynb
import os
import sys

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../credentials/sticky-notes1-data-access.json'

In [14]:
# Test that the package works
! python3 -m trainer.task \
    --job-dir ../train-output \
    --data_dir ../data/processed/tfrecords/100-examples

2020-04-29 20:58:14.005515: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-29 20:58:14.022030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f86285f4150 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-29 20:58:14.022058: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Train for 1 steps, validate for 1 steps
2020-04-29 20:58:25.111442: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
1/1 [==============================] - 20s 20s/step - loss: 6.0054 - accuracy: 0.5312 - val_loss: 12.8405 - val_accuracy: 0.6250


In [17]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
! gcloud config set ml_engine/local_python $(which python3)

# Run the trainer with ai-platform local train
! gcloud ai-platform local train \
    --package-path ./trainer \
    --module-name trainer.task \
    --job-dir ../train-output \
    -- \
    --data_dir ../data/processed/tfrecords/100-examples

Updated property [ml_engine/local_python].
2020-04-29 21:00:54.880603: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-29 21:00:54.896925: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe66042d710 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-29 21:00:54.896956: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Train for 2 steps, validate for 1 steps
2020-04-29 21:01:06.322637: I tensorflow/core/profiler/lib/profiler_session.cc:225] Profiler session started.
2/2 [==============================] - 31s 16s/step - loss: 12.8757 - accuracy: 0.5000 - val_loss: 4.0788 - val_accuracy: 0.6562


In [11]:
import datetime

BUCKET_NAME = 'sticky-notes1'
os_variables = {
    'BUCKET_NAME': BUCKET_NAME,
    'REGION': 'europe-west1',
    'JOB_NAME': 'sticky_job',
    'JOB_DIR': 'gs://' + BUCKET_NAME + '/training-jobs',
    'DATA_DIR': 'gs://' + BUCKET_NAME + '/training-data'
}
for key, value in os_variables.items():
    os.environ[key] = value

! export date=$(date +%y%m%d_%H%M%S); \
  gcloud ai-platform jobs submit training "${JOB_NAME}_${date}" \
    --config hptuning_config.yaml \
    --package-path trainer/ \
    --module-name trainer.task \
    --region $REGION \
    --python-version 3.7 \
    --runtime-version 2.1 \
    --job-dir "${JOB_DIR}/${date}" \
    -- \
    --epochs 5 \
    --data_dir $DATA_DIR

Job [sticky_job_200429_213635] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe sticky_job_200429_213635

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs sticky_job_200429_213635
jobId: sticky_job_200429_213635
state: QUEUED


In [17]:
# view training logs
! tensorboard --logdir $JOB_DIR

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.1.1 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [200]:
# MODEL_NAME="mynet"

# ! gcloud ai-platform models create $MODEL_NAME \
#   --regions $REGION

Created ml engine model [projects/mnist-keras-mle/models/mynet].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



In [211]:
# MODEL_VERSION="v1"

# # Get a list of directories in the `keras_export` parent directory. Then pick
# # the directory with the latest timestamp, in case you've trained multiple
# # times.
# SAVED_MODEL_PATH=$(gsutil ls $JOB_DIR/keras_export | tail -n 1)

# # Create model version based on that SavedModel directory
# gcloud ai-platform versions create $MODEL_VERSION \
#   --model $MODEL_NAME \
#   --runtime-version 1.15 \
#   --python-version 3.7 \
#   --framework tensorflow \
#   --origin $SAVED_MODEL_PATH